In [1]:
from bs4 import BeautifulSoup
import requests
import pickle

from os import listdir
import time

import re
import random

import pandas as pd
import numpy as np

from multiprocessing.pool import ThreadPool as Pool

import warnings
warnings.filterwarnings('ignore')

In [2]:
with open('data/osszes_link.pkl', 'rb') as pkl:
    osszes_link = pickle.load(pkl)

In [3]:
len(osszes_link)

57708

### Get data

In [4]:
headers = ['Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36',
           'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36',
           'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246', 
           'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36', 
           'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9', 
           'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36', 
           'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15.0) Gecko/20100101 Firefox/15.0.1',
           'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36',
           'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36 Edg/87.0.664.75']

In [5]:
def scrape_listing(url):
    
    page = requests.get(url, headers = {'User-Agent': random.choice(headers)})  
    time.sleep(1)

    if page.ok == False:
        return

    soup = BeautifulSoup(page.text)
    
    url_dict = {'variable' : 'URL', 'value' : url}
    tipus_dict = {'variable' : 'Típus', 'value'  : soup.find('div', class_ = 'digest').text.strip()}

    alap_adatok = [i.text for i in soup.find_all('div', class_ = 'parameterValues')]

    price = {'variable' : 'Ár', 'value' : alap_adatok[0]}
    alapterulet = {'variable' : 'Alapterület', 'value' : alap_adatok[1]}
    telekterulet = {'variable' : 'Telekterület', 'value' : alap_adatok[2]}
    rooms = {'variable' : 'Szoba', 'value' : alap_adatok[3]}
    
    parameter_nevek = [i.text.strip() for i in soup.find_all('dt', class_ = 'parameterName')]
    parameter_ertekek = [i.text.strip() for i in soup.find_all('dd', class_ = 'parameterValue')]
    parameterek = dict(zip(parameter_nevek, parameter_ertekek))
    
    cim = soup.find('div', listing = '[object Object]')['address']
    cim = {'variable' : 'Cím', 'value' : cim}
    
    leiras = soup.find('div', class_ = 'longDescription').text
    leiras = {'variable' : 'Leírás', 'value' : leiras}
    
    adatok = pd.DataFrame.from_dict(parameterek, orient = 'index').reset_index().rename(columns = {'index' : 'variable', 0 : 'value'})

    adatok = adatok.append(price, ignore_index = True)
    adatok = adatok.append(alapterulet, ignore_index = True)
    adatok = adatok.append(telekterulet, ignore_index = True)
    adatok = adatok.append(rooms, ignore_index = True)

    adatok = adatok.append(url_dict, ignore_index = True)
    adatok = adatok.append(tipus_dict, ignore_index = True)

    adatok = adatok.append(cim, ignore_index = True)
    adatok = adatok.append(leiras, ignore_index = True)

    adatok.replace('nincs megadva', np.nan, inplace = True)
    
    adatok.to_csv('data/collector/' + str(re.sub('[^0-9a-zA-Z]+', '_', url)) + '.csv', index = False)
    
    return adatok

#### Run on URLs

For 1000 it took 140 seconds --> for 57708 it will take 57708 / 1000 * 140 / 60 / 60 hours ~ 2 hours 15 mins

In [6]:
parsed_links = [i.replace('.csv', '') for i in listdir('data/collector/')]

remaining_links = []

for i in osszes_link:
    if str(re.sub('[^0-9a-zA-Z]+', '_', i)) not in parsed_links:
           remaining_links.append(i)

In [7]:
len(remaining_links)

5340

In [ ]:
# requests.get(remaining_links[43], headers = {'User-Agent': random.choice(headers)})

In [8]:
%%time

# collection = []

pool = Pool(4)

for link in remaining_links:
    pool.apply_async(scrape_listing, (link,)) #, callback = collection.append

pool.close()
pool.join()

Wall time: 33min 24s
